# Project Turak Daniyar

# Сбор и обработка данных (SQL)

1.1 Продажи за какие годы есть в ваших данных? <br>
<b>Запрос:</b> <br>
select <br>
<pre>min(c.purchased_at) min_purchased_date, </pre>
    max(c.purchased_at) max_purchased_date
from
<pre>final.carts c  </pre> 

<b>Ответ</b>:
январь 1, 2017, 7:29 утра |
декабрь 30, 2018, 11:29 вечера

1.2.1 Сколько клиентов покупали курсы? <br>
<b>Запрос</b>: <br>
select
<pre>count(distinct c.user_id)</pre>
from
<pre>final.carts c</pre> 
join
<pre>final.cart_items ci</pre>
<pre>on c.id = ci.cart_id</pre>
where         
<pre>c.state = 'successful' and</pre>
<pre>ci.resource_type = 'Course'</pre>
/*
    state = 'successful' значит что курс был куплен
*/

<b>Ответ:</b> 49006

1.2.2 Сколько всего есть различных курсов? <br>
<b>Запрос</b>: <br>
select
<pre>count(distinct ci.resource_id)</pre>
from
<pre>final.cart_items ci</pre>
where 
<pre>ci.resource_type = 'Course'</pre>

<b>Ответ:</b> 127

1.2.3 Каково среднее число купленных курсов на одного клиента? <br>
<b>Запрос</b>: <br>
with user_courses_cnt as <br>
( <br>
    select
    <pre>c.user_id,</pre>
    <pre>count(ci.resource_id) cnt_courses</pre> 
    from 
    <pre>final.carts c</pre> 
    join 
    <pre>final.cart_items ci</pre> 
    <pre>on c.id = ci.cart_id</pre>
    where 
    <pre>c.state = 'successful' and</pre>
    <pre>ci.resource_type = 'Course'</pre>
    group by 1 <br>
)<br>
select 
    <pre>avg(user_courses_cnt.cnt_courses)</pre>
from
    <pre>user_courses_cnt</pre>
    
<b>Ответ:</b> 1.44

1.2.4 Сколько клиентов купили больше одного курса? <br>
<b>Запрос</b>: <br>
with remove_duplicates as <br>
( <br>
   	select 
   	<pre>distinct c.user_id, ci.resource_id</pre>
    from 
    <pre>final.carts c</pre>
    join 
    <pre>final.cart_items ci</pre> 
    <pre>on c.id = ci.cart_id</pre>
    where 
    <pre>c.state = 'successful' and ci.resource_type = 'Course'</pre>
    order by 1 <br>
), <br>
more_than_one as <br>
( <br>
	select 
	<pre>distinct rd.user_id</pre>
  	from 
  	<pre>remove_duplicates rd</pre>
	group by 1 <br>
	having <br>
    <pre>count(rd.resource_id)>1</pre>
) <br>
select
    <pre>count(m.user_id)</pre>
from 
    <pre>more_than_one m</pre>
    
<b>Ответ:</b> 12656

# ПРОЕКТНАЯ РАБОТА

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

In [2]:
def getData():
    query = '''
    with remove_duplicates as 
    (
        select 
            distinct c.user_id, 
            ci.resource_id 
        from 
            final.carts c
        join 
            final.cart_items ci 
            on c.id = ci.cart_id
        where 
            c.state = 'successful' and ci.resource_type = 'Course'
        order by 1
    )
    select
        distinct rd.user_id,
        rd.resource_id course_id
    from 
        remove_duplicates rd
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

data = pd.DataFrame(getData())

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70424 entries, 0 to 70423
Data columns (total 2 columns):
user_id      70424 non-null int64
course_id    70424 non-null int64
dtypes: int64(2)
memory usage: 1.1 MB


In [4]:
data.head(7)

,user_id,course_id
0,169821,679
1,1239932,745
2,1141426,504
3,1221330,566
4,1663432,566
5,1137633,571
6,992335,1129


In [5]:
#Группируем по пользователям, чтобы узнать сколько пользователей
user_group = data.groupby('user_id').count().sort_values('course_id',ascending=False)

In [6]:
#Группируем по курсам, чтобы узнать сколько курсов
group_course = data.groupby('course_id').count().sort_values('user_id',ascending=False)

In [7]:
#Используя это создаем таблицу из нулей
np.zeros((len(user_group),len(group_course)))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
#В качестве индексов используем курсы а в качестве колонок пользователей
df = pd.DataFrame(np.zeros((len(group_course),len(user_group))),index=group_course.index,columns=user_group.index)
df.head()

user_id,694146,1050532,953401,722252,992470,109035,698038,565025,1017561,190140,...,1161398,1161409,1161475,1161412,1161420,1161439,1161442,1161458,1161461,2191100
course_id,,,,,,,,,,,,,,,,,,,,,
551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
794,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def insert_course(column,dataframe_to_insert,dataframe):
    #функция для того чтобы вставить 1 на месте пересечени курса и пользователя
    list_of_courses = dataframe[dataframe.user_id == column.name].course_id
    return dataframe_to_insert.index.isin(list_of_courses).astype(int)

In [10]:
df1 = df.copy()
#вставляем единички
df1 = df1.apply(lambda x: insert_course(x,df,data),axis=0)

In [11]:
df1.describe()

user_id,694146,1050532,953401,722252,992470,109035,698038,565025,1017561,190140,...,1161398,1161409,1161475,1161412,1161420,1161439,1161442,1161458,1161461,2191100
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,...,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,0.150794,0.142857,0.142857,0.134921,0.134921,0.126984,0.126984,0.126984,0.126984,0.119048,...,0.007937,0.007937,0.007937,0.007937,0.007937,0.007937,0.007937,0.007937,0.007937,0.007937
std,0.359276,0.351324,0.351324,0.343003,0.343003,0.334284,0.334284,0.334284,0.334284,0.325137,...,0.089087,0.089087,0.089087,0.089087,0.089087,0.089087,0.089087,0.089087,0.089087,0.089087
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
df1.head()

user_id,694146,1050532,953401,722252,992470,109035,698038,565025,1017561,190140,...,1161398,1161409,1161475,1161412,1161420,1161439,1161442,1161458,1161461,2191100
course_id,,,,,,,,,,,,,,,,,,,,,
551,0,1,1,1,1,1,0,1,1,1,...,1,0,0,0,0,1,0,0,0,0
566,1,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
490,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
515,1,0,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#Меняем колонки и ряды местами чтобы найти похожие курсы
df2= df1.T
df2.head()

course_id,551,566,490,794,515,745,514,840,489,507,...,836,1198,830,911,833,902,1182,1201,1199,1200
user_id,,,,,,,,,,,,,,,,,,,,,
694146,0,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1050532,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
953401,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
722252,1,1,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
992470,1,0,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
course1200 = df2[1200] #Делаем тестовый с одним курсом
course_like1200 = df2.corrwith(course1200) #считаем похожесть
gg = pd.DataFrame(course_like1200, columns=['Correlation']) #сохраняем

In [15]:
gg.head()

,Correlation
course_id,
551,-0.003146
566,-0.002435
490,-0.001544
794,0.012982
515,-0.001333


In [16]:
#добавляем колонку с количеством покупок
gg = gg.join(group_course['user_id'])

In [17]:
#Удаляем наш курс и сортируем похожие
gg.sort_values('Correlation',ascending=False).drop(1200).head()

,Correlation,user_id
course_id,,
866,0.213133,11
1104,0.063780,122
794,0.012982,2518
1199,-0.000041,2
1201,-0.000065,5


In [18]:
#Теперь выбираем только те у кого больше 100 покупок gg[gg.user_id>100] 
gg[gg.user_id>100].sort_values('Correlation', ascending=False).drop(1200, errors='ignore').head()

,Correlation,user_id
course_id,,
1104,0.063780,122
794,0.012982,2518
1187,-0.000296,105
1188,-0.000302,109
1101,-0.000306,112


In [19]:
#Выбираем первые 2 из списка похожих
[1200,gg[gg.user_id>100].sort_values('Correlation', ascending=False).drop(1200, errors='ignore'). \
 index[0],gg[gg.user_id>100].sort_values('Correlation', ascending=False). \
 drop(1200, errors='ignore').index[1]]

[1200, 1104, 794]

In [20]:
#добавляем в нашу табличку
fin_df = pd.DataFrame(data=[[1200,gg[gg.user_id>100].sort_values('Correlation', ascending=False). \
        drop(1200, errors='ignore').index[0],gg[gg.user_id>100].sort_values('Correlation', ascending=False). \
        drop(1200, errors='ignore').index[1]]],columns=['Course','Rec_Course_1','Rec_Course_2'])
fin_df.head()

,Course,Rec_Course_1,Rec_Course_2
0,1200,1104,794


In [21]:
def recommendation_table(dataframe):
    # функция которая делает все выше описанное только для каждого курса
    fin_df = pd.DataFrame(columns=['Course','Rec_Course_1','Rec_Course_2'])
    for course in dataframe.columns:
        similar_courses=df2.corrwith(dataframe[course])
        top_sim_courses = pd.DataFrame(similar_courses, columns=['Correlation'])
        top_sim_courses = top_sim_courses.join(group_course['user_id'])
        rec_courses = top_sim_courses[top_sim_courses.user_id>100].drop(course, errors='ignore').sort_values('Correlation', ascending=False).index.values[:2]
        rec_courses = np.insert(rec_courses,0,course)
        fin_df = fin_df.append(pd.DataFrame([rec_courses],columns=['Course','Rec_Course_1','Rec_Course_2']),ignore_index=True)
    return fin_df

In [22]:
fin_df = df2.pipe(recommendation_table)#Применяем нашу функцию

In [23]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(fin_df)

,Course,Rec_Course_1,Rec_Course_2
0,551,552,777
1,566,909,743
2,490,743,809
3,794,1101,1103
4,515,489,523
5,745,553,516
6,514,515,502
7,840,569,572
8,489,515,523
9,507,752,570
